In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc5_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [8]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [11]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 123
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [13]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-09 13:55:35,151] A new study created in memory with name: no-name-ad2394d0-113c-4af9-a6f2-622a76c737a1



2. Starting Optuna study...


Best trial: 0. Best value: 0.513761:   1%|          | 1/100 [00:03<05:01,  3.04s/it]

[I 2025-11-09 13:55:38,193] Trial 0 finished with value: 0.5137614678899083 and parameters: {'learning_rate': 0.04399414140994055, 'depth': 8, 'l2_leaf_reg': 0.01752493467327922, 'subsample': 0.9, 'random_strength': 0.0014595330664878767, 'bagging_temperature': 0.8597467653415687, 'border_count': 51, 'scale_pos_weight': 1.0899928448611056}. Best is trial 0 with value: 0.5137614678899083.


Best trial: 1. Best value: 0.531682:   2%|▏         | 2/100 [00:07<05:53,  3.60s/it]

...

[I 2025-11-09 13:58:47,979] Trial 97 finished with value: 0.575487012987013 and parameters: {'learning_rate': 0.0766161856967292, 'depth': 4, 'l2_leaf_reg': 0.4443869759374218, 'subsample': 0.8, 'random_strength': 0.008208450300138078, 'bagging_temperature': 0.7909338589023995, 'border_count': 255, 'scale_pos_weight': 1.5178163249815055}. Best is trial 31 with value: 0.6020444131124427.


Best trial: 31. Best value: 0.602044:  99%|█████████▉| 99/100 [03:13<00:01,  1.46s/it]

[I 2025-11-09 13:58:49,115] Trial 98 finished with value: 0.589873417721519 and parameters: {'learning_rate': 0.14824169356379488, 'depth': 5, 'l2_leaf_reg': 2.269428706482178, 'subsample': 1.0, 'random_strength': 0.07933371016480331, 'bagging_temperature': 0.25918917438730416, 'border_count': 195, 'scale_pos_weight': 2.8475789519257306}. Best is trial 31 with value: 0.6020444131124427.


Best trial: 31. Best value: 0.602044: 100%|██████████| 100/100 [03:15<00:00,  1.95s/it]

[I 2025-11-09 13:58:50,296] Trial 99 finished with value: 0.5979933110367893 and parameters: {'learning_rate': 0.10805861885699221, 'depth': 4, 'l2_leaf_reg': 6.931946231901031, 'subsample': 0.7, 'random_strength': 0.17322136286018303, 'bagging_temperature': 0.8865748038506424, 'border_count': 202, 'scale_pos_weight': 2.479648321568238}. Best is trial 31 with value: 0.6020444131124427.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.6020
  Best Hyperparameters:
    learning_rate: 0.15804097887036375
    depth: 3
    l2_leaf_reg: 0.22946926051906394
    subsample: 1.0
    random_strength: 0.8852579089082552
    bagging_temperature: 0.8538903062862053
    border_count: 207
    scale_pos_weight: 2.1219229945474476


In [14]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.15804097887036375, 'depth': 3, 'l2_leaf_reg': 0.22946926051906394, 'subsample': 1.0, 'random_strength': 0.8852579089082552, 'bagging_temperature': 0.8538903062862053, 'border_count': 207, 'scale_pos_weight': 2.1219229945474476}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.6020

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.82      0.86      4165
   Class 1.0       0.53      0.69      0.60      1235

    accuracy                           0.79      5400
   macro avg       0.72      0.76      0.73      5400
weighted avg       0.82      0.79      0.80      5400



In [15]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6020.cbm")

print("Done.")

Saving best_model...
Done.


In [16]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 161

Top 10 Most Important Features:
                        feature  importance
58            liab_prct_squared   14.854113
152  recovery_feasibility_score   10.996216
62                 liab_inverse    8.988131
61                liab_prct_log    6.934850
60               liab_prct_sqrt    5.346997
63         liab_inverse_squared    5.229173
51           witness_x_multicar    4.556612
46                liab_x_police    3.137194
53          multicar_x_highrisk    2.836996
59              liab_prct_cubed    2.705797

Bottom 10 Least Important Features:
                feature  importance
107   very_high_mileage         0.0
97    mid_price_vehicle         0.0
105        high_mileage         0.0
104  is_compact_vehicle         0.0
42        police_binary         0.0
102   is_medium_vehicle         0.0
101       medium_weight         0.0
100       light_vehicle         0.0
98      economy_vehicle         0.0
160     annual_income_z       

In [17]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            0
2       4655051            0
3       6728725            1
4       9848460            1


In [18]:
prediction.to_csv("results/catboost_6020_prediction.csv", index=False)